In [1]:
%matplotlib widget

In [2]:
from aiidalab_sssp.inspect.subwidgets.periodic_table import PeriodicTable
from aiidalab_sssp.inspect.subwidgets.select import PseudoSelectWidget
from aiidalab_sssp.inspect import DB_FOLDER
from pathlib import Path
from IPython.display import display

In [3]:
# If cached DB folder not exist (first time app run), create folder
DB_FOLDER.mkdir(parents=True, exist_ok=True)
ptable = PeriodicTable(cache_folder=str(DB_FOLDER))

display(ptable)

PeriodicTable(children=(PTableWidget(allElements=['H', 'He', 'Li', 'Be', 'B', 'C', 'N', 'O', 'F', 'Ne', 'Na', …

In [4]:
pseudo_select = PseudoSelectWidget()

# ipw.dlink(
#     (periodic_table, 'selected_element'), 
#     (pseudo_select_widget, 'selected_element'),
# )
pseudo_select.element = 'O'

display(pseudo_select)

ValueError: not enough values to unpack (expected 7, got 6)